In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

# 1. Cargar y agregar datos por producto y mes
df = pd.read_csv("sell-in.txt", sep="\t")
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")
df = df.groupby(["product_id", "periodo"])["tn"].sum().reset_index()
df = df.sort_values(["product_id", "periodo"])

# 2. Crear features (lags y mes)
def crear_features(data):
    data = data.copy()
    data["month"] = data["periodo"].dt.month
    for lag in range(1, 4):
        data[f"lag_{lag}"] = data.groupby("product_id")["tn"].shift(lag)
    return data

df_feat = crear_features(df)

# 3. Leer universo de productos
productos_objetivo = pd.read_csv("product_id_apredecir201912.txt")["product_id"].tolist()

# 4. Inicializar
resultados = []
entrenados = 0
fallbacks = 0

# 5. Loop por producto
for producto in productos_objetivo:
    data_prod = df_feat[df_feat["product_id"] == producto].copy()
    train = data_prod[data_prod["periodo"] <= "2019-12-01"]
    train = train.dropna(subset=["lag_1", "lag_2", "lag_3"])

    try:
        if train.shape[0] >= 2:
            X_train = train[["month", "lag_1", "lag_2", "lag_3"]]
            y_train = train["tn"]

            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)

            model = Ridge(alpha=10)
            model.fit(X_train_scaled, y_train)

            # Obtener lags para febrero
            dic = data_prod.loc[data_prod["periodo"] == "2019-12-01", "tn"]
            nov = data_prod.loc[data_prod["periodo"] == "2019-11-01", "tn"]
            octu = data_prod.loc[data_prod["periodo"] == "2019-10-01", "tn"]

            if not dic.empty and not nov.empty and not octu.empty:
                X_pred = pd.DataFrame([[2, dic.values[0], nov.values[0], octu.values[0]]],
                                      columns=["month", "lag_1", "lag_2", "lag_3"])
                X_pred_scaled = scaler.transform(X_pred)
                pred = model.predict(X_pred_scaled)[0]
                resultados.append({"product_id": producto, "tn_predicho": pred})
                entrenados += 1
                continue

    except Exception as e:
        print(f"⚠️ Fallo producto {producto}: {e}")

    # Fallback combinado
    data_hist = df[(df["product_id"] == producto) & (df["periodo"] < "2020-01-01")].copy()
    ultimos_12 = data_hist.sort_values("periodo").tail(12)
    prom_ultimos12 = ultimos_12["tn"].mean() if not ultimos_12.empty else 0.0
    febreros = data_hist[data_hist["periodo"].dt.month == 2]
    prom_febreros = febreros["tn"].mean() if not febreros.empty else 0.0
    pred = 0.6 * prom_ultimos12 + 0.4 * prom_febreros

    resultados.append({"product_id": producto, "tn_predicho": pred})
    fallbacks += 1

# 6. Exportar
df_resultado = pd.DataFrame(resultados)
df_resultado.to_csv("predicciones_febrero2020_ridge_780_combinado_AGREGADO.csv", index=False)

print("✅ Archivo generado: predicciones_febrero2020_ridge_780_combinado_AGREGADO.csv")
print(f"🟢 Modelos entrenados: {entrenados}")
print(f"🟡 Fallbacks aplicados: {fallbacks}")
print(f"🔢 Total productos: {len(df_resultado)}")


✅ Archivo generado: predicciones_febrero2020_ridge_780_combinado_AGREGADO.csv
🟢 Modelos entrenados: 771
🟡 Fallbacks aplicados: 9
🔢 Total productos: 780
